# Detailed API Introduction

In [1]:
import sys
sys.path.insert(0, '../data') # Paul: add directory holding API to pythonpath
import gresearch_crypto
import pandas as pd
import os

You can only submit from Kaggle Notebooks.

## TL;DR: End-to-End Usage Example